# Memory steering · Core module walkthrough

A quick tour of helper APIs in `src/memory_steering/memory_steering.py`. We compute the dimensionless groups, build a junction graph, and sample steering transitions.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, preview_json

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
try:
    from src.memory_steering.memory_steering import (
        compute_dimensionless_groups,
        y_junction_adjacency,
        transition_probs,
        sample_hop,
    )
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from src.memory_steering.memory_steering import (
        compute_dimensionless_groups,
        y_junction_adjacency,
        transition_probs,
        sample_hop,
    )

import numpy as np

Theta, Da, Lam, Gam = compute_dimensionless_groups(eta=2.0, M0=0.3, gamma=0.6, R0=0.8, T=1.0, delta=0.2, kappa=0.05, L_scale=1.0)
print(preview_json({"Theta": Theta, "D_a": Da, "Lambda": Lam, "Gamma": Gam}))

A, J, a0, b0 = y_junction_adjacency(4, 4, 4)
m = np.zeros(A.shape[0])
m[a0] = 0.7
m[b0] = -0.7
neigh = np.where(A[J] > 0)[0]
probs = transition_probs(J, neigh, m, theta=Theta)
print(preview_json({"neighbors": neigh.tolist(), "probs": probs.tolist()}))
choice = sample_hop(J, neigh, m, theta=Theta, rng=np.random.default_rng(0))
print(f"Sampled hop from junction: {choice}")


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(5, 3))
plt.bar([str(n) for n in neigh], probs, color='tab:blue')
plt.xlabel('Neighbor index')
plt.ylabel('Transition probability')
plt.title('Transition distribution at junction')
plt.ylim(0, 1)
plt.tight_layout()
plt.show()


## Notebook checklist

* The dimensionless groups (Θ, Dₐ, Λ, Γ) drive every acceptance harness; adjust the inputs to match your regime.
* `y_junction_adjacency` and `transition_probs` make it easy to prototype routing behaviours before launching large experiments.
* `sample_hop` demonstrates the stochastic selection; supply your own RNG for reproducible studies.
